In [1]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=730)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('BTC-USD', 
                      start=start_date, 
                      end=end_date, 
                      progress=False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close",  "Volume"]]
data.reset_index(drop=True, inplace=True)
data.head()

C:\Users\amlan\AppData\Local\Temp\ipykernel_29980\2733075182.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('BTC-USD',


Price,Date,Open,High,Low,Close,Volume
Ticker,,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
0,2023-11-01,34657.273438,35527.929688,34170.691406,35437.253906,22446272005
1,2023-11-02,35441.578125,35919.843750,34401.574219,34938.242188,20998158544
2,2023-11-03,34942.472656,34942.472656,34133.441406,34732.324219,17158456701
3,2023-11-04,34736.324219,35256.031250,34616.691406,35082.195312,9561294264
4,2023-11-05,35090.011719,35340.339844,34594.242188,35049.355469,12412743996


In [2]:
print(data.iloc[-1])


Price   Ticker 
Date               2025-10-30 00:00:00
Open    BTC-USD               110066.0
High    BTC-USD          111541.734375
Low     BTC-USD           106398.71875
Close   BTC-USD          107475.914062
Volume  BTC-USD            70673702912
Name: 729, dtype: object


In [3]:
data.shape

(730, 6)

In [4]:
print(data.isnull().sum())

Price   Ticker 
Date               0
Open    BTC-USD    0
High    BTC-USD    0
Low     BTC-USD    0
Close   BTC-USD    0
Volume  BTC-USD    0
dtype: int64


In [5]:
data["Date"] = pd.to_datetime(data["Date"])

In [6]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from datetime import date, timedelta

# Get data for last 2 years
today = date.today()
start_date = (today - timedelta(days=730)).strftime("%Y-%m-%d")

data = yf.download('BTC-USD', start=start_date, end=today, progress=False)

# Flatten if MultiIndex
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.droplevel(1)

# Clean up
data = data.dropna(subset=["Open", "High", "Low", "Close"])
data["Date"] = pd.to_datetime(data.index)

# Plot
figure = go.Figure(data=[go.Candlestick(
    x=data["Date"],
    open=data["Open"],
    high=data["High"],
    low=data["Low"],
    close=data["Close"]
)])
figure.update_layout(title="Bitcoin Price Analysis",
                     xaxis_rangeslider_visible=False)
figure.show()


C:\Users\amlan\AppData\Local\Temp\ipykernel_29980\502854770.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('BTC-USD', start=start_date, end=today, progress=False)


In [7]:
correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

Price
Close     1.000000
High      0.998774
Low       0.998450
Open      0.997040
Date      0.935247
Volume    0.581013
Name: Close, dtype: float64


In [9]:
from autots import AutoTS
import pandas as pd

# Ensure Date is a column (not index)
data = data.reset_index(drop=True)

# Initialize and train AutoTS model
model = AutoTS(
    forecast_length=30,  # predict next 30 days
    frequency='infer',
    ensemble='simple',
    max_generations=5,      # instead of default 25
    num_validations=2
)
model = model.fit(
    data,
    date_col='Date',
    value_col='Close',
    id_col=None
)

# Generate forecast
prediction = model.predict()
forecast = prediction.forecast

# Reset index and rename columns for cleaner output
forecast = forecast.reset_index()
forecast.columns = ['Date', 'Close']

# Display nicely formatted forecast
print(forecast)


Using 3 cpus for n_jobs.
Data frequency is: D, used frequency is: D
Model Number: 1 with model AverageValueNaive in generation 0 of 5
Model Number: 2 with model AverageValueNaive in generation 0 of 5
Model Number: 3 with model AverageValueNaive in generation 0 of 5
Model Number: 4 with model DatepartRegression in generation 0 of 5
Model Number: 5 with model DatepartRegression in generation 0 of 5
Model Number: 6 with model DatepartRegression in generation 0 of 5


c:\Users\amlan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



Model Number: 7 with model DatepartRegression in generation 0 of 5
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 7 in generation 0: DatepartRegression
Model Number: 8 with model ETS in generation 0 of 5


c:\Users\amlan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:545: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



Model Number: 9 with model ETS in generation 0 of 5
Model Number: 10 with model GLM in generation 0 of 5
Model Number: 11 with model GLM in generation 0 of 5
Model Number: 12 with model GLS in generation 0 of 5
Model Number: 13 with model GLS in generation 0 of 5
Model Number: 14 with model LastValueNaive in generation 0 of 5
Model Number: 15 with model LastValueNaive in generation 0 of 5
Model Number: 16 with model LastValueNaive in generation 0 of 5
Model Number: 17 with model LastValueNaive in generation 0 of 5
Model Number: 18 with model SeasonalNaive in generation 0 of 5
Model Number: 19 with model SeasonalNaive in generation 0 of 5
Model Number: 20 with model SeasonalNaive in generation 0 of 5
Model Number: 21 with model VAR in generation 0 of 5
Template Eval Error: ValueError('Only gave one variable to VAR') in model 21 in generation 0: VAR
Model Number: 22 with model VAR in generation 0 of 5
Template Eval Error: ValueError('Only gave one variable to VAR') in model 22 in generat

...

31 - Ensemble with avg smape 2.45: 
32 - Ensemble with avg smape 2.37: 
33 - Ensemble with avg smape 3.88: 
34 - Ensemble with avg smape 2.52: 
35 - Ensemble with avg smape 3.12: 
36 - Ensemble with avg smape 3.05: 
Template Eval Error: Exception("Transformer HistoricValues failed on fit from params fake_date {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 1.0, 'first_value_only': False, 'threshold': 10, 'threshold_method': 'mean'}, '1': {'window': 28}} with error KeyError('2')") in model 36 in generation 0: Ensemble
Model Number: 37 of 9 with model Ensemble for Validation 2


c:\Users\amlan\AppData\Local\Programs\Python\Python312\Lib\site-packages\autots\tools\anomaly_utils.py:1266: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



37 - Ensemble with avg smape 2.36: 
38 - Ensemble with avg smape 2.3: 
39 - Ensemble with avg smape 3.17: 
40 - Ensemble with avg smape 2.5: 
41 - Ensemble with avg smape 3.1: 
42 - Ensemble with avg smape 3.24: 
Template Eval Error: Exception("Transformer HistoricValues failed on fit from params fake_date {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 1.0, 'first_value_only': False, 'threshold': 10, 'threshold_method': 'mean'}, '1': {'window': 28}} with error KeyError('2')") in model 42 in generation 0: Ensemble
Model Number: 43 of 9 with model Ensemble for Validation 2
43 - Ensemble with avg smape 2.5: 
44 - Ensemble with avg smape 2.45: 
45 - Ensemble with avg smape 3.31: 
46 - Ensemble with avg smape 2.51: 
47 - Ensemble with avg smape 3.11: 
48 - Ensemble with avg smape 3.23: 
Template Eval Error: Exception("Transformer HistoricValues failed on fit from params fake_date {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 1.0, 'first_value_only': False, 'th

c:\Users\amlan\AppData\Local\Programs\Python\Python312\Lib\site-packages\autots\evaluator\auto_model.py:3068: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

c:\Users\amlan\AppData\Local\Programs\Python\Python312\Lib\site-packages\autots\evaluator\auto_model.py:3068: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

c:\Users\amlan\AppData\Local\Programs\Python\Python312\Lib\site-packages\autots\tools\anomaly_utils.py:1266: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a futur

         Date          Close
0  2025-10-31  107475.914062
1  2025-11-01  110406.043750
2  2025-11-02  110823.029167
3  2025-11-03  111825.431771
4  2025-11-04  112631.305208
5  2025-11-05  113372.000000
6  2025-11-06  113993.393975
7  2025-11-07  115005.530513
8  2025-11-08  115894.022396
9  2025-11-09  116394.230208
10 2025-11-10  116709.071875
11 2025-11-11  116729.963021
12 2025-11-12  115306.369616
13 2025-11-13  114276.417757
14 2025-11-14  113727.557008
15 2025-11-15  113292.033157
16 2025-11-16  113022.899219
17 2025-11-17  113044.980469
18 2025-11-18  112240.294488
19 2025-11-19  111637.586207
20 2025-11-20  111480.550854
21 2025-11-21  111375.624719
22 2025-11-22  111251.468144
23 2025-11-23  111419.119112
24 2025-11-24  111671.358854
25 2025-11-25  111620.162535
26 2025-11-26  111602.503125
27 2025-11-27  111505.752947
28 2025-11-28  112018.368750
29 2025-11-29  112413.428125
